In [11]:
import sqlite3
import os
import json
import pandas as pd

In [4]:
def from_json(file_name):
    try:
        result = False
        with open(file_name, "r") as f:
            result = json.load(f)
        return result
    except Exception as e:
        print(e)
        return result   

In [3]:
list_of_bib_objects = from_json("bib_objects.json")

In [4]:
list_of_bib_objects

[{'Title': 'Complex dynamical states in binary mixture convection with weak negative Soret coupling',
  'Author': 'Arantxa Alonso , Oriol Batiste , Alvaro Meseguer,, Isabel Mercader',
  'Journal': 'PHYSICAL REVIEW E',
  'Year': '2007',
  'Pages': '026310-1-026310-15',
  'Volume': '75',
  'Owner': 'che',
  'Timestamp': '2014.09.05'},
 {'Title': 'Traveling wave convection patterns in an annular cell',
  'Author': ' Anderson, K. E. , Behringer, R. P. ',
  'Journal': 'Physica D',
  'Year': '1991',
  'Pages': '444-449',
  'Volume': '51'},
 {'Title': 'Long time scales in traveling wave convection patterns ',
  'Author': ' Anderson, K. E. , Behringer, R. P. ',
  'Journal': 'Phys. Rev. A.',
  'Year': '1990',
  'Pages': '323',
  'Volume': '145'},
 {'Title': ' Numerical Solution of Boundary Value Problems for Ordinary DifferentialEquations, Prentice Hall Series in Computational Mathematicsl ',
  'Author': ' Ascher, U. M. , Mattheij, R. M. M. , Russell, R.D.',
  'Journal': 'Phys. Rev. Lett',
  'Y

In [5]:
SQL_CREATE_TABLE_AUTHORS = '''
CREATE TABLE IF NOT EXISTS Authors (
author_id INTEGER PRIMARY KEY,
author_name TEXT DEFAULT NULL
);'''

In [6]:
SQL_CREATE_TABLE_JOURNALS = '''
CREATE TABLE IF NOT EXISTS Journals (
journal_id INTEGER PRIMARY KEY,
journal_title TEXT DEFAULT NULL
);'''

In [7]:
SQL_CREATE_TABLE_ARTICLES = '''
CREATE TABLE IF NOT EXISTS Articles (
article_id INTEGER PRIMARY KEY,
article_title TEXT DEFAULT NULL,
journal_id INTEGER,
year DATETIME DEFAULT NULL,
pages TEXT DEFUALT NULL,
volume TEXT DEFAULT NULL,
FOREIGN KEY (journal_id) REFERENCES Journals(journal_id)
);'''

In [8]:
SQL_CREATE_TABLE_AUTHORS_ARTICLES = '''
CREATE TABLE IF NOT EXISTS AuthorsArticles (
author_id INTEGER NOT NULL,
article_id INTEGER NOT NULL,
FOREIGN KEY (author_id) REFERENCES Authors(author_id),
FOREIGN KEY (article_id) REFERENCES Articles(article_id)
);'''

In [9]:
SQL_CREATE_UNIQUE_INDEX_AUTHOR = '''
CREATE UNIQUE INDEX author_index ON Authors (author_name);
'''

In [18]:
def execute_sql(SQL, db_name, data=[]):
    try:
        connection = sqlite3.connect(db_name)
        cursor = connection.cursor()
        cursor.execute(SQL, data)
        connection.commit()
        connection.close()
        return True
    except sqlite3.Error as e:
        print(e)
        return False

In [11]:
DB_NAME = os.path.join('/Users/maksimsadkov/lab1', 'main_db.db')
DB_NAME

'/Users/maksimsadkov/lab1/main_db.db'

In [12]:
execute_sql(SQL_CREATE_UNIQUE_INDEX_AUTHOR, DB_NAME)

index author_index already exists


False

In [13]:
execute_sql(SQL_CREATE_TABLE_AUTHORS, DB_NAME)

True

In [14]:
execute_sql(SQL_CREATE_TABLE_JOURNALS, DB_NAME)

True

In [15]:
execute_sql(SQL_CREATE_TABLE_ARTICLES, DB_NAME)

True

In [16]:
execute_sql(SQL_CREATE_TABLE_AUTHORS_ARTICLES, DB_NAME)

True

In [17]:
def insert_values(bib_objects, db_name):
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    
    for obj in bib_objects:
        author_id = None
        article_id = None
        journal_id = None
        if 'Journal' in obj.keys():
            SQL = '''INSERT INTO Journals (journal_title) VALUES(?);'''
            data = [obj['Journal'].strip()]
            cur.execute(SQL, data)
            journal_id = cur.lastrowid
        if journal_id is not None:
            SQL = '''INSERT INTO Articles (article_title, journal_id, year, pages, volume) VALUES(?, ?, ?, ?, ?);'''
            data = []
            if 'Title' in obj.keys():
                data.append(obj['Title'].strip())
            else:
                data.append(None)
                    
            data.append(journal_id)
            
            if 'Year' in obj.keys():
                data.append(obj['Year'].strip())
            else:
                data.append(None)
    
            if 'Pages' in obj.keys():
                data.append(obj['Pages'].strip())
            else:
                data.append(None)
    
            if 'Volume' in obj.keys():
                data.append(obj['Volume'].strip())
            else:
                data.append(None)
                    
            cur.execute(SQL, data)
            data = []
            article_id = cur.lastrowid
        if article_id is not None:
            SQL_INSERT_AUTHOR = '''INSERT OR IGNORE INTO Authors(author_name) VALUES(?);'''
            SQL_INSERT_ARTICLE_AUTHOR = '''INSERT INTO AuthorsArticles(author_id, article_id) VALUES(?, ?);'''
            if 'Author' in obj.keys():
                authors_data = obj['Author'].split(' , ')
                for author in authors_data:
                    data = [author.strip()]
                    cur.execute(SQL_INSERT_AUTHOR, data)
                    author_id = cur.lastrowid
                    cur.execute(SQL_INSERT_ARTICLE_AUTHOR, [author_id, article_id])
    conn.commit()
    conn.close()
                

In [18]:
#insert_values(list_of_bib_objects, DB_NAME)

In [31]:
def get_df(SQL, db_name):
    connection = sqlite3.connect(db_name)
    df = pd.read_sql_query(SQL, connection)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    return df

In [32]:
author_df = get_df("SELECT * from Authors ORDER BY author_name;", DB_NAME)

In [33]:
df = get_df('''SELECT author_name, article_title from AuthorsArticles
            LEFT JOIN Authors on Authors.author_id = AuthorsArticles.author_id
            lEFT JOIN Articles on Articles.article_id = AuthorsArticles.article_id
            ORDER BY author_name;''', DB_NAME)

In [34]:
df

,author_name,article_title
0,None,Hydrodynamic instabilities in ferronematics
1,None,Influence of sidementation on convective instabilities in colloidal suspensions
2,None,Magnetic-field-driven instability in stratified ferrofluids
3,None,Thermal convection in colloidal suspensions with negative separation ratio
4,None,Onset of convection in colloids stratified by gravity
5,None,Phys. Rev. E
6,None,Onset of convection in colloids stratified by gravity
7,None,Convection instability of magnetized ferrofluids
8,None,The onset of thermomagnetic convection in stratified frerrofluids
9,None,Traveling-wave convection in colloids stratified by gravity


In [35]:
author_df

,author_id,author_name
0,37,A. A. Bozhko
1,41,A. Bozhko
2,70,"Ahlers, G."
3,21,"Alonso, A."
4,3,"Alvaro Meseguer,, Isabel Mercader"
5,4,"Anderson, K. E."
6,1,Arantxa Alonso
7,6,"Ascher, U. M."
8,88,"Aubry, N."
9,9,Author1


In [36]:
article_df = get_df("SELECT * from Articles ORDER BY article_title;", DB_NAME)

In [37]:
article_df

,article_id,article_title,journal_id,year,pages,volume
0,85,"A review on nanofluids: Preparation, stability mechanisms, and applications",85,2012,435873,None
1,60,An hysteresis loop in the two component Bernard problem,60,1975,1071-1075,18
2,86,Average Volumetric Concentration in Two-Phase Flow Systems,86,1965,453468,87
3,74,Binary-fluid-mixture convection with low-frequency modulated heating,74,2010,016310,82
4,40,Bistability of moving and self-pinned fronts of supercritical localized convection structures,40,2007,14002(6),80
5,26,Bistable Hate Transfer in a Nanofluid,26,2009,10503(4),102
6,46,Characterization of dispersive chaos and related states of binary-fluid convection,46,1995,165–224,85
7,19,Classification of Spatially Localized Oscillations in Periodically Forced DissipativeSystems,19,2012,665-711,7
8,43,Coexisting traveling waves and steady rolls in binary-fluid convection,43,1993,R665–R668,48
9,78,Collisions of localized convection structures in binary fluid mixtures,78,2012,093055,14


In [38]:
article_df = get_df("SELECT * from Journals ORDER BY journal_title;", DB_NAME)

In [40]:
article_df

,journal_id,journal_title
0,22,"10-th International Meeting on Thermodiffusion 4-8 June 2012, Brussels, Belgium, Abstr."
1,103,11-я Зимняя школа по механике сплошных сред.
2,40,EPL
3,84,EPL
4,64,European Physical Journal E
5,51,Fluid Dyn. Res.
6,28,Fluid Mech
7,55,Fluid Mech.
8,60,Heat Mass Transfere
9,17,Heat Transfer Research


In [ ]:
# Напишем тесты для основных функций

In [1]:
import unittest

In [19]:
class TestLab1Functions(unittest.TestCase):
    def test_from_json(self):
        result = from_json("file.json")
        self.assertEqual(result, [1,2,3])
        result = from_json("bib_objects.json")
        self.assertIsInstance(result, list)
        result = from_json("a")
        self.assertFalse(result)

    def test_execute_sql(self):
        DB_NAME = '/Users/maksimsadkov/lab1/main_db.db'
        result = execute_sql("SELECT * FROM Articles", DB_NAME)
        self.assertTrue(result)
        result = execute_sql("SELECT * FROM Anybody", DB_NAME)
        self.assertFalse(result)
        

In [20]:
unittest.main(argv=[''], verbosity=3, exit=False)

test_execute_sql (__main__.TestLab1Functions.test_execute_sql) ... ok
test_from_json (__main__.TestLab1Functions.test_from_json) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


no such table: Anybody
[Errno 2] No such file or directory: 'a'
